## 1. 전처리(Preprocessing)
- Tokenizer()
    - 토큰화와 정수 인코딩에 사용
    - 훈련 데이터로부터 단어 집합을 생성, 해당 단어 집합으로부터 임의의 문장을 정수 인코딩 해줌

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
train_text = 'The earth is an awesome place to live'

# 단어 집합 생성
tokenizer.fit_on_texts([train_text])     

# 정수 인코딩
sub_text = "The earth is an great place to live"
# 2차원이 결과로 나오기 때문에 [0] 추가해주기

sequences = tokenizer.texts_to_sequences([sub_text])[0]

print("정수 인코딩 : ", sequences)
# great 는 단어 집합(vocab)에 없기 때문에 출력되지 않는ㄷ.ㅏ
print("단어 집합 : ", tokenizer.word_index)

정수 인코딩 :  [1, 2, 3, 4, 6, 7, 8]
단어 집합 :  {'the': 1, 'earth': 2, 'is': 3, 'an': 4, 'awesome': 5, 'place': 6, 'to': 7, 'live': 8}


- pad_sequence() 
    - 전체 훈련 데이터에서 각 샘플의 길이는 서로 다를 수 있다. 따라서 각 데이터의 길이를 맞추어야 하는데 이를 패딩이라고 한다.
    - 보통 숫자 0을 넣어서 다른 샘플의 길이를 맞추어준다.
    - pre 는 앞부터, Post는 뒤부터 0을 채워준다

In [7]:
pad_sequences([[1,2,3], [3,4,5,6], [7,8]], maxlen=3, padding='pre')

array([[1, 2, 3],
       [4, 5, 6],
       [0, 7, 8]], dtype=int32)

## 2. Word Embedding
- 텍스트 내의 단어들을 밀집 벡터(dense vector)로 만드는 것
    - 원-핫 벡터에 비해 상대적으로 저차원을 가지며, 모든 원소의 값이 실수이다.
    - Ex) [0.1 -1.2 0.8 0.2 1.8] # 상대적으로 저차원이며 실수값을 가짐

- 임베딩 벡터는 초기에는 랜덤값을 가지지만, 인공 신경망의 가중치가 학습되는 방법과 같은 방식으로 값이 학습되며 변동된다.

![](2022-02-06-13-40-43.png)

- Embedding()
    - 단어를 밀집 벡터로 만들어줌
    - (number of samples, input_length)인 2D 정수 텐서를 입력받음
        - 각 sample 은 정수 인코딩이 된 결과 -> 정수의 시퀀스
    - 워드 임베딩 작업을 수행하고, (number of samples, input_length, embedding word dimensionality)인 3D 텐서를 리턴

In [ ]:
# 1. 토큰화 
tokenized_text = [['Hope', 'to', 'see', 'you', 'soon'], ['Nice', 'to', 'see', 'you', 'again']]

# 2. 각 단어에 대한 정수 인코딩
encoded_text = [[0, 1, 2, 3, 4],[5, 1, 2, 3, 6]]

# 3. 위 정수 인코딩 데이터가 아래의 임베딩 층의 입력이 된다.
vocab_size = 7
embedding_dim = 2
# Embedding - (단어 집합의 크기, 임베딩 벡터의 출력 차원, 입력 시퀀스의 길이)
Embedding(vocab_size, embedding_dim, input_length=5)


# 각 정수는 아래의 테이블의 인덱스로 사용되며 Embedding()은 각 단어마다 임베딩 벡터를 리턴한다.
+------------+------------+
|   index    | embedding  |
+------------+------------+
|     0      | [1.2, 3.1] |
|     1      | [0.1, 4.2] |
|     2      | [1.0, 3.1] |
|     3      | [0.3, 2.1] |
|     4      | [2.2, 1.4] |
|     5      | [0.7, 1.7] |
|     6      | [4.1, 2.0] |
+------------+------------+

# 위의 표는 임베딩 벡터가 된 결과를 예로서 정리한 것이고 Embedding()의 출력인 3D 텐서를 보여주는 것이 아님.                                                                                                                                                                                                                                                                                     

## 3. 모델링(Modeling)
- Sequential()에 임베딩 층 추가하기

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding

model = Sequential()
model.add(Embedding(vocab_size, output_dim, input_length))

Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2022-02-06 17:04:12.367557: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-06 17:04:12.367677: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


NameError: name 'vocab_size' is not defined

## 4. 함수형 모델(Functional API)
- 각 층을 일종의 함수로 정의한다
- 각 함수를 조합하기 위한 연산자들을 활용하여 신경망을 설계한다.


#### 1) 전결합 피드 포워드 신경망(Fully-connected FFNN)
- Sequential API와는 다르게 입력 데이터의 크기(shape)를 인자로 입력층을 정의해야 한다.

In [9]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model



# 10개의 입력을 받는 입력층 만들기
inputs = Input(shape=(10,))
hidden1 = Dense(64, activation='relu')(inputs)
hidden2 = Dense(64, activation='relu')(hidden1)
output = Dense(1, activation='sigmoid')(hidden2)

model = Model(inputs=inputs, outputs=output)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


# 은닉층과 출력층의 변수를 통일해도 됨
inputs = Input(shape=(10,))
x = Dense(8, activation="relu")(inputs)
x = Dense(4, activation="relu")(x)
x = Dense(1, activation="linear")(x)
model = Model(inputs, x)



#### 2) 선형 회귀(Linear Regression)

In [10]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model

X = [1,2,3,4,5,6,7,8,9]
y = [11,22,33,44,53,66,77,87,95]

inputs = Input(shape=(1,))
output = Dense(1, activation='linear')(inputs)
linear_model = Model(inputs, output)

sgd = optimizers.SGD(lr=0.01)

linear_model.compile(optimizer=sgd, loss='mse', metrics=['mse'])
linear_model.fit(X, y , epochs=300)

/Users/sunny/miniforge3/envs/py38/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
2022-02-06 17:14:44.803272: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-02-06 17:14:44.897140: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/300
1/1 [==============================] - 1s 870ms/step - loss: 4411.8408 - mse: 4411.8408
Epoch 2/300
1/1 [==============================] - 0s 5ms/step - loss: 543.9653 - mse: 543.9653
Epoch 3/300
1/1 [==============================] - 0s 5ms/step - loss: 68.0505 - mse: 68.0505
Epoch 4/300
1/1 [==============================] - 0s 6ms/step - loss: 9.4913 - mse: 9.4913
Epoch 5/300
1/1 [==============================] - 0s 6ms/step - loss: 2.2844 - mse: 2.2844
Epoch 6/300
1/1 [==============================] - 0s 6ms/step - loss: 1.3961 - mse: 1.3961
Epoch 7/300
1/1 [==============================] - 0s 6ms/step - loss: 1.2853 - mse: 1.2853
Epoch 8/300
1/1 [==============================] - 0s 6ms/step - loss: 1.2702 - mse: 1.2702
Epoch 9/300
1/1 [==============================] - 0s 5ms/step - loss: 1.2668 - mse: 1.2668
Epoch 10/300
1/1 [==============================] - 0s 6ms/step - loss: 1.2649 - mse: 1.2649
Epoch 11/300
1/1 [==============================] - 0s 5ms/step -

#### 3) 로지스틱 회귀(Logistic Regression)

In [11]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

inputs = Input(shape=(3,))
output = Dense(1, activation='sigmoid')(inputs)
logistic_model = Model(inputs, output)

#### 4) 다중 입력을 받는 모델
- 함수형 API 를 사용하면 다중 입력,출력을 가지는 모델을 만들 수 있다.


In [ ]:
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model

# 두 개의 입력층을 정의
inputA = Input(shape=(64,))
inputB = Input(shape=(128,))

# 첫번째 입력층으로부터 분기되어 진행되는 인공 신경망을 정의
x = Dense(16, activation="relu")(inputA)
x = Dense(8, activation="relu")(x)
x = Model(inputs=inputA, outputs=x)

# 두번째 입력층으로부터 분기되어 진행되는 인공 신경망을 정의
y = Dense(64, activation="relu")(inputB)
y = Dense(32, activation="relu")(y)
y = Dense(8, activation="relu")(y)
y = Model(inputs=inputB, outputs=y)

# 두개의 인공 신경망의 출력을 연결(concatenate)
result = concatenate([x.output, y.output])

z = Dense(2, activation="relu")(result)
z = Dense(1, activation="linear")(z)

model = Model(inputs=[x.input, y.input], outputs=z)

#### 5) RNN(Recurrence Neural Network) 은닉층 사용하기

In [ ]:
from tensorflow.keras.layers import Input, Dense, LSTM
from tensorflow.keras.models import Model

inputs = Input(shape=(50,1))
lstm_layer = LSTM(10)(inputs)
x = Dense(10, activation='relu')(lstm_layer)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=output)

## 5. 케라스 서브클래싱 API(Keras Subclassing API)
- 케라스 구현 방식
    - Sequential
    - Functional
    - Subclassing

In [12]:
# 선형회귀를 Subclassing API 로 구현하기
import tensorflow as tf

class LinearRegression(tf.keras.Model):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear_layer = tf.keras.layers.Dense(1, input_dim=1, activation='linear')
    
    def call(self, x):
        y_pred = self.linear_layer(x)
        
        return y_pred

In [ ]:
model = LinearRegression()

X = [1, 2, 3, 4, 5, 6, 7, 8, 9] # 공부하는 시간
y = [11, 22, 33, 44, 53, 66, 77, 87, 95] # 각 공부하는 시간에 맵핑되는 성적

sgd = tf.keras.optimizers.SGD(lr=0.01)
model.compile(optimizer=sgd, loss='mse', metrics=['mse'])
model.fit(X, y, epochs=300)

#### ***** 3가지 구현 방식 비교 *****


- 1) Sequential API

    - 장점 : 단순하게 층을 쌓는 방식으로 쉽고 사용하기가 간단합니다.
    - 단점 : 다수의 입력(multi-input), 다수의 출력(multi-output)을 가진 모델 또는 층 간의 연결(concatenate)이나 덧셈(Add)과 같은 연산을 하는 모델을 구현하기에는 적합하지 않습니다. 이런 모델들의 구현은 Functional API를 사용해야 합니다.

- 2) Functional API

    - 장점 : Sequential API로는 구현하기 어려운 복잡한 모델들을 구현할 수 있습니다.
    - 단점 : 입력의 크기(shape)를 명시한 입력층(Input layer)을 모델의 앞단에 정의해주어야 합니다. 가령, 아래의 코드를 봅시다.

    
```python
# 선형 회귀 구현 코드의 일부 발췌
inputs = Input(shape=(1,)) # <-- 해당 부분
output = Dense(1, activation='linear')(inputs)
linear_model = Model(inputs, output)

sgd = optimizers.SGD(lr=0.01)

linear_model.compile(optimizer=sgd, loss='mse', metrics=['mse'])
linear_model.fit(X, y, epochs=300)
```
- 3) Subclassing API

    - 장점 : Functional API로도 구현할 수 없는 모델들조차 구현이 가능합니다.
    - 단점 : 객체 지향 프로그래밍(Object-oriented programming)에 익숙해야 하므로 코드 사용이 가장 까다롭습니다.